# data scafold


In [ ]:
from itertools import filterfalse

# Simulace DNA scaffoldů s různou kvalitou
scaffold_data = [
    {"id": "scaffold_001", "length": 15000, "gc_content": 0.42, "n_count": 50, "coverage": 30.5},
    {"id": "scaffold_002", "length": 8000, "gc_content": 0.38, "n_count": 1200, "coverage": 5.2},
    {"id": "scaffold_003", "length": 25000, "gc_content": 0.45, "n_count": 100, "coverage": 45.0},
    {"id": "scaffold_004", "length": 3000, "gc_content": 0.28, "n_count": 800, "coverage": 2.1},
    {"id": "scaffold_005", "length": 18000, "gc_content": 0.41, "n_count": 75, "coverage": 28.7},
]

# 1. Filtrování scaffoldů s vysokou kvalitou
def is_high_quality(scaffold):
    """Vrátí True pokud scaffold splňuje kritéria kvality"""
    return (
        scaffold["length"] >= 5000 and          # Minimální délka
        scaffold["n_count"] <= 200 and          # Maximálně 200 neznámých bází (N)
        scaffold["coverage"] >= 10.0 and        # Minimální coverage
        0.35 <= scaffold["gc_content"] <= 0.50  # GC obsah v rozumném rozsahu
    )

# Scaffoldy, které NEmají vysokou kvalitu (potenciální problémy)
low_quality = list(filterfalse(is_high_quality, scaffold_data))
print("Scaffoldy s nízkou kvalitou:")
for s in low_quality:
    print(f"  {s['id']}: délka={s['length']}, N={s['n_count']}, coverage={s['coverage']}")
# Výstup: scaffold_002, scaffold_004

# 2. Identifikace potenciálních kontaminací
def is_probably_bacterial(scaffold):
    """Identifikace bakteriálních kontaminací podle GC obsahu"""
    return 0.25 <= scaffold["gc_content"] <= 0.75  # Typické bakteriální rozmezí

def is_probably_host_dna(scaffold):
    """Identifikace hostitelské DNA (např. lidské)"""
    return 0.40 <= scaffold["gc_content"] <= 0.42  # Typické pro lidskou DNA

# Co NENÍ ani bakteriální, ani hostitelské DNA? (zajímavé neznámé sekvence)
unknown_origin = list(filterfalse(
    lambda s: is_probably_bacterial(s) or is_probably_host_dna(s),
    scaffold_data
))
print("\nScaffoldy neznámého původu:")
for s in unknown_origin:
    print(f"  {s['id']}: GC={s['gc_content']:.2%}")
# Výstup: scaffold_004 (GC=28% - možná archea nebo jiná kontaminace)

# 3. Filtrace scaffoldů pro draft genome assembly
def suitable_for_draft_genome(scaffold):
    """Scaffold vhodný pro draft genome assembly"""
    return scaffold["length"] >= 10000 and scaffold["coverage"] >= 20.0

# Scaffoldy NEvhodné pro draft genome (pouze pro doplňkové analýzy)
supplementary_scaffolds = list(filterfalse(suitable_for_draft_genome, scaffold_data))
print("\nScaffoldy pro doplňkové analýzy (ne draft genome):")
for s in supplementary_scaffolds:
    print(f"  {s['id']}: délka={s['length']}, coverage={s['coverage']}")

# Pokročilé použití v bioinformatice:

In [ ]:
from itertools import filterfalse, islice
from collections import Counter

# Simulace scaffoldů z FASTA souboru
class DNAScaffold:
    def __init__(self, header, sequence):
        self.id = header.split()[0][1:]  # Odstranění '>'
        self.sequence = sequence.upper()
        self.length = len(sequence)
        self.gc_content = self.calculate_gc()
        self.n_count = sequence.upper().count('N')
        
    def calculate_gc(self):
        seq = self.sequence
        gc_count = seq.count('G') + seq.count('C')
        total = len(seq) - seq.count('N')  # Ignorovat neznámé báze
        return gc_count / total if total > 0 else 0
    
    def __repr__(self):
        return f"Scaffold({self.id}, len={self.length}, GC={self.gc_content:.2%})"

# Načtení scaffoldů (zjednodušené)
scaffolds = [
    DNAScaffold(">scaffold_1", "ATGCNATGCTANNCGTAGCTANNN"),
    DNAScaffold(">scaffold_2", "ATGCTAGCTAGCTAGCTAGCTAGC"),
    DNAScaffold(">scaffold_3", "NNNNNNNNNNNNNNNNNNNNNNNN"),
    DNAScaffold(">scaffold_4", "GCGCGCGCGCGCGCGCGCGCGCGC"),
    DNAScaffold(">scaffold_5", "ATATATATATATATATATATATAT"),
]

# 1. Odstranění scaffoldů s vysokým obsahem neznámých bází (N)
def too_many_unknowns(scaffold, threshold=0.1):
    """Vrátí True pokud má scaffold příliš mnoho neznámých bází"""
    return scaffold.n_count / scaffold.length > threshold

clean_scaffolds = list(filterfalse(too_many_unknowns, scaffolds))
print(f"Čisté scaffoldy (N < 10%): {len(clean_scaffolds)}/{len(scaffolds)}")

# 2. Identifikace možných chimér nebo artefaktů
def is_potential_chimera(scaffold):
    """Detekce možných chimér - náhlé změny v k-mer distribuci"""
    if scaffold.length < 100:
        return False
    
    # Zjednodušená detekce - rozdělení na části a porovnání GC obsahu
    part1 = scaffold.sequence[:scaffold.length//2]
    part2 = scaffold.sequence[scaffold.length//2:]
    
    gc1 = (part1.count('G') + part1.count('C')) / len(part1)
    gc2 = (part2.count('G') + part2.count('C')) / len(part2)
    
    # Pokud se GC obsah výrazně liší, možná chimera
    return abs(gc1 - gc2) > 0.15

non_chimeric = list(filterfalse(is_potential_chimera, clean_scaffolds))

# 3. Filtrace pro různé typy analýz
def suitable_for_gene_prediction(scaffold):
    """Kritéria pro predikci genů"""
    return (
        scaffold.length >= 1000 and
        scaffold.n_count == 0 and
        0.35 <= scaffold.gc_content <= 0.65
    )

def suitable_for_repeat_analysis(scaffold):
    """Kritéria pro analýzu repetitivních sekvencí"""
    return scaffold.length >= 500 and scaffold.gc_content < 0.3

# Scaffoldy nevhodné pro predikci genů (ale vhodné pro jiné analýzy)
non_gene_scaffolds = list(filterfalse(suitable_for_gene_prediction, scaffolds))

# Scaffoldy nevhodné pro analýzu repetitivních sekvencí
non_repeat_scaffolds = list(filterfalse(suitable_for_repeat_analysis, scaffolds))

# Praktický workflow pro analýzu metagenomických scaffoldů:

In [ ]:
from itertools import filterfalse, chain
import statistics

class ScaffoldAnalyzer:
    def __init__(self, scaffolds):
        self.scaffolds = scaffolds
    
    def filter_by_coverage(self, min_coverage=5.0):
        """Filtruje scaffoldy s nízkým coverage"""
        return list(filterfalse(lambda s: s['coverage'] < min_coverage, self.scaffolds))
    
    def filter_by_completeness(self, max_n_percentage=0.05):
        """Filtruje scaffoldy s vysokým obsahem gapů (N)"""
        return list(filterfalse(
            lambda s: s.get('n_count', 0) / s['length'] > max_n_percentage,
            self.scaffolds
        ))
    
    def identify_organelle_dna(self):
        """Identifikace organelové DNA (mitochondrie, chloroplasty)"""
        def is_organelle_like(scaffold):
            # Organelová DNA často má specifický GC obsah a k-mer profil
            gc = scaffold['gc_content']
            length = scaffold['length']
            
            # Mitochondriální DNA často nižší GC a specifická délka
            is_mitochondrial = (0.35 <= gc <= 0.45 and 15000 <= length <= 20000)
            # Chloroplastová DNA
            is_chloroplast = (0.35 <= gc <= 0.38 and 120000 <= length <= 160000)
            
            return is_mitochondrial or is_chloroplast
        
        return list(filterfalse(is_organelle_like, self.scaffolds))
    
    def find_rare_scaffolds(self, coverage_threshold_percentile=10):
        """Najde scaffoldy s neobvykle nízkým nebo vysokým coverage"""
        coverages = [s['coverage'] for s in self.scaffolds]
        low_threshold = statistics.quantiles(coverages, n=100)[coverage_threshold_percentile-1]
        high_threshold = statistics.quantiles(coverages, n=100)[100-coverage_threshold_percentile]
        
        def is_extreme_coverage(scaffold):
            cov = scaffold['coverage']
            return cov < low_threshold or cov > high_threshold
        
        return list(filterfalse(is_extreme_coverage, self.scaffolds))
    
    def analyze_strand_bias(self, max_bias=0.7):
        """Filtruje scaffoldy se silným strand biasem (možné artefakty)"""
        def has_strand_bias(scaffold):
            forward = scaffold.get('forward_coverage', 0)
            reverse = scaffold.get('reverse_coverage', 0)
            total = forward + reverse
            
            if total == 0:
                return False
            
            bias = max(forward, reverse) / total
            return bias > max_bias  # Příliš velká preference jednoho strandu
        
        return list(filterfalse(has_strand_bias, self.scaffolds))

# Použití
sample_data = [
    {'id': 's1', 'length': 15234, 'gc_content': 0.43, 'n_count': 12, 'coverage': 25.1,
     'forward_coverage': 13, 'reverse_coverage': 12.1},
    {'id': 's2', 'length': 8231, 'gc_content': 0.37, 'n_count': 450, 'coverage': 3.2,
     'forward_coverage': 2.9, 'reverse_coverage': 0.3},  # Silný strand bias!
    {'id': 's3', 'length': 16500, 'gc_content': 0.38, 'n_count': 8, 'coverage': 18.5,
     'forward_coverage': 9.2, 'reverse_coverage': 9.3},
]

analyzer = ScaffoldAnalyzer(sample_data)

print("Scaffoldy s dobrou kvalitou:")
good_quality = analyzer.filter_by_completeness(max_n_percentage=0.1)
good_quality = analyzer.filter_by_coverage(min_coverage=5.0)
print([s['id'] for s in good_quality])  # ['s1', 's3']

print("\nScaffoldy bez extrémního strand biasu:")
no_bias = analyzer.analyze_strand_bias(max_bias=0.8)
print([s['id'] for s in no_bias])  # ['s1', 's3'] - s2 má bias 2.9/(2.9+0.3)=0.91

# Typické použití při scaffold analýze:

In [ ]:
# Pipeline pro čištění scaffoldů před assemblí
def scaffold_cleaning_pipeline(scaffolds):
    from itertools import filterfalse
    
    # Postupné aplikování filtrů
    cleaned = scaffolds
    
    # 1. Odstranit velmi krátké scaffoldy
    cleaned = filterfalse(lambda s: s.length < 500, cleaned)
    
    # 2. Odstranit scaffoldy s příliš mnoho neznámých bází
    cleaned = filterfalse(lambda s: s.n_count > s.length * 0.1, cleaned)
    
    # 3. Odstranit možné kontaminace (extrémní GC obsah)
    cleaned = filterfalse(lambda s: s.gc_content < 0.2 or s.gc_content > 0.8, cleaned)
    
    # 4. Odstranit scaffoldy s nízkým coverage
    cleaned = filterfalse(lambda s: s.coverage < 5.0, cleaned)
    
    return list(cleaned)

In [ ]:
from itertools import compress

produkty = ['jablka', 'hrušky', 'banány', 'pomeranče', 'jahody']
ceny = [25, 30, 35, 40, 50]
skladem = [True, False, True, True, False]

# Vybereme produkty, které jsou skladem a stojí méně než 40 Kč
vyber = list(compress(
    produkty, 
    [cena < 40 and dostupnost for cena, dostupnost in zip(ceny, skladem)]
))
print("Dostupné a levné produkty:", vyber)  # Výstup: ['jablka', 'banány']

In [3]:
import base64
import json
from IPython.display import Image, display

def mm(graph):
    # 1. Nejlepší praxe pro mermaid.ink je zabalit graf do JSONu
    graph_data = {"code": graph, "mermaid": {"theme": "default"}}
    json_str = json.dumps(graph_data)
    
    # 2. Kódování do UTF-8 bajtů
    graphbytes = json_str.encode("utf8")
    
    # 3. Použití URL-safe Base64 kódování (nahradí + a / za - a _)
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("utf8")
    
    # 4. Služba mermaid.ink vyžaduje odstranění paddingu (=) na konci
    # i když to moderní prohlížeče často zvládnou, je lepší to vyčistit
    # (nicméně urlsafe_b64encode je hlavní klíč k úspěchu)
    
    display(
        Image(
            url="https://mermaid.ink/img/" + base64_string
        )
    )

# Váš graf (Mermaid syntaxe je v pořádku)
mm("""
graph TD
    A["count() generátor"] --> B["Generuje čísla"]
    B --> C["0 → 1 → 2 → 3 → 4 → 5 → 6 → ..."]
    
    C --> D["Výraz: i % 2 == 0"]
    
    D --> E["Vyhodnocení pro každé i"]
    E --> F["0 % 2 == 0 → True"]
    E --> G["1 % 2 == 0 → False"]
    E --> H["2 % 2 == 0 → True"]
    E --> I["3 % 2 == 0 → False"]
    E --> J["4 % 2 == 0 → True"]
    E --> K["5 % 2 == 0 → False"]
    E --> L["... pokračuje donekonečna"]
    
    F --> M["Selektory: True, False, True, False, True, False, ..."]
    G --> M
    H --> M
    I --> M
    J --> M
    K --> M
    L --> M
    
    N["Data: 'nula', 'jedna', 'dva', 'tři', 'čtyři', 'pět'"] --> O["compress párování"]
    M --> O
    
    O --> P["True + 'nula' → 'nula' ✓"]
    O --> Q["False + 'jedna' → přeskočeno"]
    O --> R["True + 'dva' → 'dva' ✓"]
    O --> S["False + 'tři' → přeskočeno"]
    O --> T["True + 'čtyři' → 'čtyři' ✓"]
    O --> U["False + 'pět' → přeskočeno"]
    
    P --> V["Výsledek: 'nula'"]
    R --> V
    T --> V
    
    V --> W["Konečný seznam: ['nula', 'dva', 'čtyři']"]
""")
